In [227]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 

In [263]:
def convert_price(datas) : 
    return_val = 0
    temp = "0"
    for data in datas : 
        if data == "억" :
            return_val += int(temp) * 10000
            temp = "0"
        
        elif data == " " or data == "," :
            continue
        else : 
            temp += data
    return return_val + int(temp)
            

In [264]:
driver = webdriver.Chrome("./chromedriver")
driver.get("https://new.land.naver.com/offices?ms=37.579225,126.9368,15&a=SG:SMS:GJCG:APTHGJ:GM:TJ&e=RETAIL")
where = "서울시 대방동"

search_box = driver.find_element_by_xpath('//*[@id="search_input"]')
search_box.send_keys(where)

In [265]:
find_button = driver.find_element_by_xpath('//*[@id="wrap"]/header/div[1]/form/fieldset/button[1]')
find_button.click()

In [266]:
sanga_btn = driver.find_element_by_xpath('//*[@id="wrap"]/div[1]/a[4]')
sanga_btn.click()

In [267]:
html = driver.page_source


In [268]:
soup = BeautifulSoup(html, "html.parser")

In [269]:
lists = soup.find_all("div",attrs="item_list item_list--article")[0].find_all("div",attrs="item false")

In [270]:
data_table = pd.DataFrame(columns=["이름","유형","담보/매매가","월세","유형","크기","층", "방향","상세설명","위치"])
data_table

,이름,유형,담보/매매가,월세,유형,평수,층,방향,상세설명,위치


In [271]:

ls = []


for l in lists : 
  

    datas = l.find("div")
    title = datas.find("div",attrs="item_title").text
    types = datas.find("span",attrs="type").text
    price_deposit = convert_price(datas.find("span",attrs="price").text.split("/")[0])
    
    
    # 전/월세
    try : 
        price_monthly = convert_price(datas.find("span",attrs="price").text.split("/")[1])
        
    # 매매
    except : 
        price_monthly = 0
    info_type = datas.find("strong", attrs="type").text
    
    # 방향정보 있을 때
    if len(datas.find("span",attrs="spec").text.split(",")) == 3 : 
        info_area, info_floor, info_direction = datas.find("span",attrs="spec").text.split(",")
        
    # 방향정보 없을 때 
    else : 
        info_area, info_floor = datas.find("span",attrs="spec").text.split(",")
        info_direction = "방향정보없음"
        
    info_detail = datas.find("div",attrs="info_area").find_all("span")[1].text
        
    ls.append([title,types,price_deposit,price_monthly,info_type,info_area, info_floor, info_direction,info_detail,where])

    
driver.close()

----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------
----------------------


In [272]:
for l in ls : 
    data_table.loc[len(data_table)] = l

## data_table